This Notebook file is used to calculate the transformation parameters for a Lidar. 

Every LIDAR is fixed to one of the rigid robot links. The readings the lidars give can be mapped to real world coordinates with the formula
\begin{equation}
p_{base} = T^{base}_{joint}(q) \cdot T^{joint}_{lidar} \cdot p_{lidar}
\end{equation}

TODO: More explanation and derivation of the formula

\begin{equation}
p_{base} = T^{base}_{joint}(q) \cdot T^{joint}_{lidar} \cdot p_{lidar}
\end{equation}

In [197]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from utils import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
msmts = pd.DataFrame(pd.read_pickle('/home/dlrc1/measurements/20180919T1532290000_ALL.pkl'))
msmts['lidar3'] = msmts['lidar_data'].apply(lambda x: x[3])

In [199]:
# get median / avg
print(msmts['lidar3'].mean())
print(msmts['lidar3'].median())
print(msmts['state_j_pos'][0])

71.62911611785096
72.0
[-0.06284837  0.3468821  -0.06420437 -2.24170899 -0.01354111  1.52779067
  0.02296058]


Now we need to calculate the known values for every time step, which are:<br>
The sensor readings $ s_i $ <br>
The inverted rotation from the joint transformation $R^{base}_{joint}(q)^{-1}=R^{joint}_{base}(q)=R_i^T = R_i^{-1}$<br>
The translation vector of the transformation matrices $P^{base}_{joint}(q) = P_i$


In [200]:
rotations, translations = [], []
lidarvalues = []
for i in np.arange(10,150,2):
    Tproduct, Tlist, Tjoint, coord_EE = get_jointToCoordinates(msmts.iloc[i]['state_j_pos'].tolist(), trueCoordinates=None, untilJoint=5)
    lidar3 = msmts.iloc[i]['lidar3']/1000
    print(i)
    print('lidar3', lidar3)
    print('coord_EE', coord_EE)
    print('Tjoint', Tjoint)
    print('')
    lidarvalues.append(lidar3)
    rotations.append(Tjoint[:3,:3])
    translations.append(Tjoint[:3,-1])

predicted coordinates [ 0.30738123 -0.04540963  0.13628907  1.        ]
10
lidar3 0.073
coord_EE [ 0.30738123 -0.04540963  0.13628907  1.        ]
Tjoint [[-0.84715271  0.11143598  0.51953278  0.45341877]
 [ 0.07973599  0.99335045 -0.0830485  -0.0564713 ]
 [-0.52533271 -0.0289293  -0.85040498  0.31900107]
 [ 0.          0.          0.          1.        ]]

predicted coordinates [ 0.30738231 -0.04540984  0.13629035  1.        ]
12
lidar3 0.073
coord_EE [ 0.30738231 -0.04540984  0.13629035  1.        ]
Tjoint [[-0.84715019  0.11143715  0.51953664  0.45341886]
 [ 0.07973673  0.99335033 -0.08304928 -0.05647158]
 [-0.52533667 -0.02892906 -0.85040255  0.31900314]
 [ 0.          0.          0.          1.        ]]

predicted coordinates [ 0.30738131 -0.04540989  0.13628974  1.        ]
14
lidar3 0.071
coord_EE [ 0.30738131 -0.04540989  0.13628974  1.        ]
Tjoint [[-0.84715172  0.11143718  0.51953414  0.45341849]
 [ 0.07973685  0.99335031 -0.0830493  -0.05647168]
 [-0.52533418 -0.0289293

predicted coordinates [ 0.30738142 -0.04541114  0.13628927  1.        ]
86
lidar3 0.076
coord_EE [ 0.30738142 -0.04541114  0.13628927  1.        ]
Tjoint [[-0.84715185  0.11144067  0.51953318  0.45341867]
 [ 0.07973924  0.99334989 -0.08305213 -0.05647323]
 [-0.52533361 -0.02893058 -0.85040438  0.31900148]
 [ 0.          0.          0.          1.        ]]

predicted coordinates [ 0.3073828  -0.04541059  0.13629043  1.        ]
88
lidar3 0.069
coord_EE [ 0.3073828  -0.04541059  0.13629043  1.        ]
Tjoint [[-0.84714957  0.11143868  0.51953732  0.45341929]
 [ 0.0797376   0.99335014 -0.08305073 -0.05647243]
 [-0.52533753 -0.02892973 -0.85040199  0.31900327]
 [ 0.          0.          0.          1.        ]]

predicted coordinates [ 0.30738183 -0.04541022  0.13628958  1.        ]
90
lidar3 0.078
coord_EE [ 0.30738183 -0.04541022  0.13628958  1.        ]
Tjoint [[-0.8471514   0.11143795  0.51953449  0.45341877]
 [ 0.07973728  0.99335022 -0.08305004 -0.05647202]
 [-0.52533462 -0.0289296

In [201]:
n = len(lidarvalues)
stacks = []
for i in range(n):
    longmatrix = (eval("np.zeros((3,3)),"*i + "-rotations[i].T" + ", np.zeros((3,3))"*(n-i-1)))
    longmatrix = np.concatenate(longmatrix)
    args = (np.eye(3), lidarvalues[i]*np.eye(3), longmatrix)
    stacks.append(np.concatenate(args))
    print(stack)
    print('')

[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.47        0.          0.        ]
 [ 0.          0.47        0.        ]
 [ 0.          0.          0.47      ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.97503662 -0.2207513  -0.02392605]
 [ 0.02544433 -0.00403642

[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.47        0.          0.        ]
 [ 0.          0.47        0.        ]
 [ 0.          0.          0.47      ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.97503662 -0.2207513  -0.02392605]
 [ 0.02544433 -0.00403642

[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.47        0.          0.        ]
 [ 0.          0.47        0.        ]
 [ 0.          0.          0.47      ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.97503662 -0.2207513  -0.02392605]
 [ 0.02544433 -0.00403642

[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.47        0.          0.        ]
 [ 0.          0.47        0.        ]
 [ 0.          0.          0.47      ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.97503662 -0.2207513  -0.02392605]
 [ 0.02544433 -0.00403642

[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.47        0.          0.        ]
 [ 0.          0.47        0.        ]
 [ 0.          0.          0.47      ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.97503662 -0.2207513  -0.02392605]
 [ 0.02544433 -0.00403642

In [202]:
stacks = tuple(stacks)
bigmatrix = np.concatenate(stacks, axis=1).T
print(bigmatrix.shape)

(210, 216)


In [203]:
indixes_unknown = list(range(6))
indixes_known = []
for i in range(n):
    indixes_unknown += [i*3+6]
    indixes_known += [i*3+8]
for i in range(n):
    indixes_unknown += [i*3+7]
print(indixes_unknown)
print(indixes_known)
A_unknown = bigmatrix[:, indixes_unknown]
A_known = bigmatrix[:, indixes_known]
print(A_unknown.shape)
print(A_known.shape)

[0, 1, 2, 3, 4, 5, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97, 100, 103, 106, 109, 112, 115, 118, 121, 124, 127, 130, 133, 136, 139, 142, 145, 148, 151, 154, 157, 160, 163, 166, 169, 172, 175, 178, 181, 184, 187, 190, 193, 196, 199, 202, 205, 208, 211, 214]
[8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98, 101, 104, 107, 110, 113, 116, 119, 122, 125, 128, 131, 134, 137, 140, 143, 146, 149, 152, 155, 158, 161, 164, 167, 170, 173, 176, 179, 182, 185, 188, 191, 194, 197, 200, 203, 206, 209, 212, 215]
(210, 146)
(210, 70)


In [204]:
vectors = []
for ax in range(3):
    for i in range(n):
        vectors.append(np.dot(-rotations[i][ax,:].reshape(1,-1),translations[i].reshape(3,1)))
vectors = tuple(vectors)
vectors = np.concatenate(vectors,axis=0)
print(vectors.shape)

(210, 1)


In [205]:
b_unknown = vectors[:2*n,:]
b_known = vectors[2*n:,:]
print(b_unknown.shape, b_known.shape)
print(A_unknown.shape)

(140, 1) (70, 1)
(210, 146)


In [208]:
# print(A_unknown.shape, vectors.shape)
# as we split and rearranged the matrices, the new system is:
# b_rearranged is b_unknown stacked with b_known (b_u on top)
# A_unknown * x_unknown = b_rearranged

x, residuals, rank, singval = np.linalg.lstsq(A_unknown, vectors, rcond=-1)

print(f'Rank: {rank}')
#print(f'Singular values: {singval}')
print(f'residuals: {residuals}')
print(f'Translation parameters: {x[:6]}')

Rank: 146
residuals: [0.53526576]
Translation parameters: [[  60895.43699786]
 [-164018.81844961]
 [  53220.86502794]
 [-866958.76665365]
 [2288359.35726669]
 [-753129.55904159]]
